In [1]:
# library(glmnet)
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [40]:
# tissue = args[1]
tissue = "blood"
# tissue_predOn = args[2]
tissue_predOn = "germline"
# model_name = args[3]
model_name = "model3"
tmp_file_path = "../../../"

if ((tissue_predOn == "liver")&&(tissue != "liver")){
    load(paste(tmp_file_path,"data/",tissue,"/objects/",model_name,"/",tissue,"_onLiver_model.RData",sep=""))#model
}else{
    load(paste(tmp_file_path,"data/",tissue,"/objects/",model_name,"/",tissue,"_model.RData",sep=""))#model
}
if ((tissue_predOn != "liver")&&(tissue == "liver")){
    load(paste(tmp_file_path,"data/",tissue_predOn,"/objects/",model_name,"/",tissue_predOn,"_onLiver_samples_sites_test.RData",sep=""))#sample_sites_test
    all_data <- read.csv(paste(tmp_file_path,"data/",tissue_predOn,"/dataframes/",model_name,"/",tissue_predOn,"_onLiver_all_data_readyForPrediction.csv",sep=""),header=TRUE)
}else{
    load(paste(tmp_file_path,"data/",tissue_predOn,"/objects/",model_name,"/",tissue_predOn,"_samples_sites_test.RData",sep=""))#sample_sites_test
    all_data <- read.csv(paste(tmp_file_path,"data/",tissue_predOn,"/dataframes/",model_name,"/",tissue_predOn,"_all_data_readyForPrediction.csv",sep=""),header=TRUE)
}
    

probs <- predict.glm(model, all_data[sample_sites_test,], type="response")
probs_df <- data.frame(x = probs)

#binding the predicted proabilities to the OG data (but the test sites only)
probs_df <- probs_df %>%
    bind_cols(all_data[sample_sites_test,])

#renaming the glm_probs coloumn
colnames(probs_df ) <- replace(colnames(probs_df ), 1, "glm_probs")

#writing to file 
filename = paste(tmp_file_path,"data/",tissue,"/dataframes/",model_name,"/",tissue,"_on_",tissue_predOn,"_ProbabilityDf.csv",sep="")#this sep is for the filename string
write.csv(probs_df ,filename, row.names = FALSE)

Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
“prediction from a rank-deficient fit may be misleading”
